In [ ]:
# Install Dependencies
!pip install fastapi uvicorn nest-asyncio pyngrok groq

# Imports
import nest_asyncio
import threading
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Dict, List
from groq import Groq
from pyngrok import ngrok
import uvicorn
import json
import os

# Fix Colab event loop issue
nest_asyncio.apply()


# Configuration
GROQ_API_KEY = ""
NGROK_AUTHTOKEN = ""

ngrok.set_auth_token(NGROK_AUTHTOKEN)

# Initialize LLM client
client = Groq(api_key=GROQ_API_KEY)


#Create FastAPI App
app = FastAPI(title="Chat API", version="1.0")


# In-Memory Storage
chat_sessions: Dict[str, List[Dict[str, str]]] = {}


# Request Schema (Validation)
class ChatRequest(BaseModel):
    session_id: str
    message: str


# Utility Functions

def build_messages(history, user_message):
    """
    Builds structured message list for Groq
    """
    return [
        {"role": "system", "content": "You are a helpful AI assistant."},
        *history,
        {"role": "user", "content": user_message}
    ]


def dump_sessions_to_json(filename="chat_sessions.json"):
    """
    Saves all sessions to JSON file
    """
    with open(filename, "w") as f:
        json.dump(chat_sessions, f, indent=4)
    print("Sessions saved to file")


# API Endpoints

@app.get("/health")
def health():
    return {"status": "Server running "}


@app.get("/sessions")
def get_sessions():
    return chat_sessions


@app.post("/chat")
def chat(request: ChatRequest):
    try:
        # Create new session if not exists
        if request.session_id not in chat_sessions:
            chat_sessions[request.session_id] = []

        history = chat_sessions[request.session_id]

        # Build structured messages
        messages = build_messages(history, request.message)

        # Call Groq model
        response = client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=messages,
            temperature=0.7
        )

        reply = response.choices[0].message.content

        # Save conversation
        history.append({"role": "user", "content": request.message})
        history.append({"role": "assistant", "content": reply})

        # Optional: Auto-save to JSON
        dump_sessions_to_json()

        return {
            "session_id": request.session_id,
            "reply": reply
        }

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


# Start Server

public_url = ngrok.connect(8000)
print("Public URL:", public_url)

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

threading.Thread(target=run).start()


Public URL: NgrokTunnel: "https://unresident-gabrielle-reproachfully.ngrok-free.dev" -> "http://localhost:8000"


In [ ]:
dump_sessions_to_json()


Sessions saved to file
